In [1]:
from collections import defaultdict
import json, math
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.options.display.float_format = '{:.2f}'.format

In [2]:
filename = './data/examples/chatbot_arena-annotation.json'
raw_data = pd.read_json(filename).sort_values(ascending=True, by=["tstamp"])
raw_data.loc[raw_data['winner'] == 'modedl_b', 'winner'] = 'model_b'

battles = raw_data[raw_data['anony']].reset_index(drop=True)
battles

,model_a,model_b,winner,judge,turn,anony,language,tstamp,source,data
0,koala-13b,gpt-4,model_b,arena_user,1,True,English,1706445022.31,koala-13b-vs-gpt-4.jsonl,"{'instruction': 'Kuka keksi psykiatrian', 'ins..."
1,gpt-4,koala-13b,model_a,arena_user,1,True,English,1706445022.31,koala-13b-vs-gpt-4.jsonl,{'instruction': 'Implementing an RNN with atte...
2,koala-13b,gpt-4,model_b,arena_user,1,True,English,1706445022.31,koala-13b-vs-gpt-4.jsonl,{'instruction': 'Write a sample function in Py...
3,gpt-4,koala-13b,model_a,arena_user,1,True,English,1706445022.31,koala-13b-vs-gpt-4.jsonl,"{'instruction': 'Ile to 2 razy 2?', 'instructi..."
4,gpt-4,koala-13b,model_a,arena_user,1,True,English,1706445022.31,koala-13b-vs-gpt-4.jsonl,{'instruction': 'Generate a sentence using zoo...
...,...,...,...,...,...,...,...,...,...,...
7466,vicuna-13b,koala-13b,tie,arena_user,1,True,English,1706445025.73,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'Do not include any non-numeri...
7467,vicuna-13b,koala-13b,tie,arena_user,1,True,English,1706445025.73,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'Write the letters in sequence...
7468,vicuna-13b,koala-13b,model_b,arena_user,1,True,English,1706445025.73,vicuna-13b-vs-koala-13b.jsonl,"{'instruction': 'Hello', 'instruction_trans': ..."
7469,koala-13b,vicuna-13b,model_b,arena_user,1,True,English,1706445025.73,vicuna-13b-vs-koala-13b.jsonl,"{'instruction': 'Hi', 'instruction_trans': '',..."


MAD selected data, using `text-embedding-ada-002` metric

In [3]:
filename = './data/examples/chatbot_arena-annotation-10-embedding.json'
raw_data = pd.read_json(filename).sort_values(ascending=True, by=["tstamp"])
raw_data.loc[raw_data['winner'] == 'modedl_b', 'winner'] = 'model_b'

battles_10_embedding = raw_data[raw_data['anony']].reset_index(drop=True)
battles_10_embedding

,model_a,model_b,winner,judge,turn,anony,language,tstamp,source,data
0,gpt-4,koala-13b,model_a,arena_user,1,True,English,1706515434.84,koala-13b-vs-gpt-4.jsonl,{'instruction': 'Write a sample function in Py...
1,koala-13b,gpt-4,model_b,arena_user,1,True,English,1706515434.84,koala-13b-vs-gpt-4.jsonl,{'instruction': 'Generate a sentence using zoo...
2,gpt-4,koala-13b,tie,arena_user,1,True,English,1706515434.84,koala-13b-vs-gpt-4.jsonl,{'instruction': 'What is the most efficient wa...
3,koala-13b,gpt-4,tie,arena_user,1,True,English,1706515434.84,koala-13b-vs-gpt-4.jsonl,{'instruction': 'Write a hilarious original jo...
4,gpt-4,koala-13b,tie,arena_user,1,True,English,1706515434.84,koala-13b-vs-gpt-4.jsonl,"{'instruction': 'faça uma fanfic +18 em ptbr',..."
...,...,...,...,...,...,...,...,...,...,...
205,koala-13b,vicuna-13b,model_b,arena_user,1,True,English,1706515434.97,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'Half of my heart is in Havana...
206,vicuna-13b,koala-13b,model_a,arena_user,1,True,English,1706515434.97,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'Fill in the blanks: my name i...
207,koala-13b,vicuna-13b,tie,arena_user,1,True,English,1706515434.97,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'у Барсика конфета у Мурзика п...
208,koala-13b,vicuna-13b,tie,arena_user,1,True,English,1706515434.97,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'Write me a poem in urdu in th...


MAD selected data, using `bert-score` metric

In [4]:
filename = './data/examples/chatbot_arena-annotation-10-bert.json'
raw_data = pd.read_json(filename).sort_values(ascending=True, by=["tstamp"])
raw_data.loc[raw_data['winner'] == 'modedl_b', 'winner'] = 'model_b'

battles_10_bert = raw_data[raw_data['anony']].reset_index(drop=True)
battles_10_bert

,model_a,model_b,winner,judge,turn,anony,language,tstamp,source,data
0,koala-13b,gpt-4,model_b,arena_user,1,True,English,1706513835.23,koala-13b-vs-gpt-4.jsonl,{'instruction': '- 너는 여러가지 신용카드를 파는 신용카드 판매원이야...
1,koala-13b,gpt-4,model_b,arena_user,1,True,English,1706513835.23,koala-13b-vs-gpt-4.jsonl,"{'instruction': '안녕?!', 'instruction_trans': '..."
2,koala-13b,gpt-4,model_b,arena_user,1,True,English,1706513835.23,koala-13b-vs-gpt-4.jsonl,{'instruction': 'Hãy làm một bài thơ về phong ...
3,gpt-4,koala-13b,tie,arena_user,1,True,English,1706513835.23,koala-13b-vs-gpt-4.jsonl,"{'instruction': 'ты говоришь по русски?', 'ins..."
4,koala-13b,gpt-4,model_b,arena_user,1,True,English,1706513835.23,koala-13b-vs-gpt-4.jsonl,"{'instruction': 'Cześć! Umiesz polski?', 'inst..."
...,...,...,...,...,...,...,...,...,...,...
205,koala-13b,vicuna-13b,tie,arena_user,1,True,English,1706513835.36,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'Beende folgenden Satz: eene m...
206,koala-13b,vicuna-13b,model_b,arena_user,1,True,English,1706513835.36,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'Напиши стих или поэму или про...
207,vicuna-13b,koala-13b,tie,arena_user,1,True,English,1706513835.36,vicuna-13b-vs-koala-13b.jsonl,"{'instruction': 'Xavier 's mom has 3 kids, Gro..."
208,vicuna-13b,koala-13b,tie,arena_user,1,True,English,1706513835.36,vicuna-13b-vs-koala-13b.jsonl,{'instruction': 'output a random json string a...


# Elo Rating

In [5]:
def compute_elo(battles, K=4, SCALE=400, BASE=10, INIT_RATING=1000):
    rating = defaultdict(lambda: INIT_RATING)

    for rd, model_a, model_b, winner in battles[['model_a', 'model_b', 'winner']].itertuples():
        ra = rating[model_a]
        rb = rating[model_b]
        ea = 1 / (1 + BASE ** ((rb - ra) / SCALE))
        eb = 1 / (1 + BASE ** ((ra - rb) / SCALE))
        if winner == "model_a":
            sa = 1
        elif winner == "model_b":
            sa = 0
        elif winner == "tie" or winner == "tie (bothbad)":
            sa = 0.5
        else:
            raise Exception(f"unexpected vote {winner}")
        rating[model_a] += K * (sa - ea)
        rating[model_b] += K * (1 - sa - eb)

    return rating

def preety_print_elo_ratings(ratings):
    df = pd.DataFrame([
        [n, elo_ratings[n]] for n in elo_ratings.keys()
    ], columns=["Model", "Elo rating"]).sort_values("Elo rating", ascending=False).reset_index(drop=True)
    df["Elo rating"] = (df["Elo rating"] + 0.5).astype(int)
    df.index = df.index + 1
    return df

def get_bootstrap_result(battles, func_compute_elo, num_round):
    rows = []
    for i in tqdm(range(num_round), desc="bootstrap"):
        rows.append(func_compute_elo(battles.sample(frac=1.0, replace=False)))
    df = pd.DataFrame(rows)
    return df[df.median().sort_values(ascending=False).index]

In [6]:
BOOTSTRAP_ROUNDS = 1000

np.random.seed(42)
bootstrap_elo_lu = get_bootstrap_result(battles, compute_elo, BOOTSTRAP_ROUNDS)
bootstrap_lu_median = bootstrap_elo_lu.median().reset_index().set_axis(["model", "Elo rating"], axis=1)
bootstrap_lu_median["Elo rating"] = (bootstrap_lu_median["Elo rating"] + 0.5).astype(int)
bootstrap_lu_median

bootstrap: 100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 104.62it/s]


,model,Elo rating
0,gpt-4,1192
1,claude-v1,1133
2,gpt-3.5-turbo,1073
3,vicuna-13b,996
4,koala-13b,935
5,alpaca-13b,848
6,chatglm-6b,824


In [7]:
# text-embedding-ada-002

BOOTSTRAP_ROUNDS = 1000

np.random.seed(42)
bootstrap_elo_lu = get_bootstrap_result(battles_10_embedding, compute_elo, BOOTSTRAP_ROUNDS)
bootstrap_lu_median = bootstrap_elo_lu.median().reset_index().set_axis(["model", "Elo rating"], axis=1)
bootstrap_lu_median["Elo rating"] = (bootstrap_lu_median["Elo rating"] + 0.5).astype(int)
bootstrap_lu_median

bootstrap: 100%|█████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1168.08it/s]


,model,Elo rating
0,gpt-4,1048
1,gpt-3.5-turbo,1012
2,claude-v1,1008
3,vicuna-13b,1002
4,koala-13b,987
5,alpaca-13b,982
6,chatglm-6b,962


In [8]:
# bert-score
BOOTSTRAP_ROUNDS = 1000

np.random.seed(42)
bootstrap_elo_lu = get_bootstrap_result(battles_10_bert, compute_elo, BOOTSTRAP_ROUNDS)
bootstrap_lu_median = bootstrap_elo_lu.median().reset_index().set_axis(["model", "Elo rating"], axis=1)
bootstrap_lu_median["Elo rating"] = (bootstrap_lu_median["Elo rating"] + 0.5).astype(int)
bootstrap_lu_median

bootstrap: 100%|█████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1446.79it/s]


,model,Elo rating
0,gpt-4,1056
1,claude-v1,1038
2,gpt-3.5-turbo,1020
3,vicuna-13b,1002
4,koala-13b,967
5,alpaca-13b,964
6,chatglm-6b,954
